In [1]:
import pandas  as pd
import   numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection   import train_test_split
import pickle
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
from sklearn.compose import ColumnTransformer

In [2]:
df =pd.read_csv('Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace =True)

In [5]:
X=df.drop(['EstimatedSalary'],axis=1)
y=df['EstimatedSalary']

In [6]:
num_features = X.select_dtypes(exclude="object").columns
onehot_columns=['Geography','Gender']
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer= StandardScaler()
oh_transformer=OneHotEncoder()
preprocessor= ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,onehot_columns),
        ("StandardScaler",numeric_transformer,num_features)
    ],remainder='passthrough'
)

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.42,random_state=42)

In [8]:

X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [9]:
with open("preprocessorreg.pkl", "wb") as file:
    pickle.dump(preprocessor, file)

In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [12]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),##HL1 connected with input layer
    Dense(32,activation='relu'),##Hl2
    Dense(1)## output layer
])

###compile the model
model.compile(optimizer='adam',loss="mean_absolute_error",metrics=['mae'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3009 (11.75 KB)
Trainable params: 3009 (11.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
## set up tensor board
log_dir="regressionlogs/fit/" +datetime.datetime.now().strftime("%Y%m%D-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
##setup early stopping
earrly_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [16]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                  callbacks=[tensorflow_callback,earrly_stopping_callback])


Epoch 1/100


182/182 [==============================] - 7s 17ms/step - loss: 101340.4219 - mae: 101340.4219 - val_loss: 98226.0312 - val_mae: 98226.0312
Epoch 2/100
182/182 [==============================] - 2s 8ms/step - loss: 100947.8047 - mae: 100947.8047 - val_loss: 97390.9375 - val_mae: 97390.9375
Epoch 3/100
182/182 [==============================] - 2s 10ms/step - loss: 99432.9688 - mae: 99432.9688 - val_loss: 95094.0234 - val_mae: 95094.0234
Epoch 4/100
182/182 [==============================] - 2s 11ms/step - loss: 96216.8516 - mae: 96216.8516 - val_loss: 90927.4922 - val_mae: 90927.4922
Epoch 5/100
182/182 [==============================] - 2s 9ms/step - loss: 91074.1328 - mae: 91074.1328 - val_loss: 84944.5000 - val_mae: 84944.5000
Epoch 6/100
182/182 [==============================] - 2s 10ms/step - loss: 84225.3516 - mae: 84225.3516 - val_loss: 77634.5078 - val_mae: 77634.5078
Epoch 7/100
182/182 [==============================] - 2s 10ms/step - loss: 76335.2109 - mae: 76

In [17]:
model.save('modelreg.h5')

c:\Users\timex\Documents\data science2024\deep learning\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
import tensorboard
%load_ext tensorboard

In [20]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 11172), started 0:00:12 ago. (Use '!kill 11172' to kill it.)

In [24]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'TEST MAE:{test_mae}')

132/132 [==============================] - 0s 3ms/step - loss: 49550.5781 - mae: 49550.5781
TEST MAE:49550.578125
